This jupyter notebook is to show function `babs-init`.

In [1]:
# See `README_howtorun_show_babs_init_ipynb.md` for how to run this file!

# Clone out:
#   datalad clone ria+file:///path/to/<container>/output_ria#~data singleORmulti-ses_<container>_outputs
# If you want to change `project_root`'s foldername, also change `output_ria/alias/data` symlink:
    # cd output_ria/alias
    # ln -sf /new/full/path/to/output_ria/xxx/xxx-xxx-xxx-xxx data

# flake8: noqa

# set up how Jupyter notebook behaves:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'   # print all outputs

import os
import os.path as op

There are several arguments when calling `babs-init`:

In [4]:
# This notebook only accepts toybidsapp_rawBIDS, qsiprep or fmriprep
#   or fmriprep_anatonly or 'fmriprep_sloppyFlag' or fmriprepfake (input ds: BIDS)
# ++++++++++++++++++++++++++++++++++
bidsapp = "fmriprep"
task_name = "sloppy"   # for fmriprep: 'anatonly', 'sloppy'; for toybidsapp: 'rawBIDS'
type_session = "single-ses"
which_dataset = "PNC"    # "toy_fake", "toy_real", "HBN", "PNC"

flag_where = "msi"     # "cubic" or "local" or "msi"
type_system = "slurm"   # "sge" or "slurm"
# ++++++++++++++++++++++++++++++++++

# sanity checks:
if flag_where == "cubic":
    assert type_system == "sge"
elif flag_where == "msi":
    assert type_system == "slurm"

# where:
if flag_where == "cubic":
    where_root = "/cbica/projects/BABS"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "local":
    where_root = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "msi":
    where_root = "/home/faird/zhaoc"
    where_project = "/home/faird/zhaoc/data"
    where_notebooks = op.join(where_root, "babs/notebooks")
else:
    raise Exception("not valid `flag_where`!")

# Input dataset:
if which_dataset == "toy_fake":
    if type_session == "multi-ses":
        input_ds = "https://osf.io/w2nu3/"
        # input_ds = op.join(where_project, "w2nu3")
    elif type_session == "single-ses":
        input_ds = 	"https://osf.io/t8urc/"
        # input_ds = op.join(where_project, "t8urc")
elif which_dataset == "toy_real":  # real data:
    if type_session == "multi-ses":
        # input_ds = "/cbica/projects/RBC/chenying_practice/data_for_babs/NKI/data_hashedID_bids"
        input_ds = "/cbica/projects/BABS/data/testdata_NKI/data_hashedID_bids"
    elif type_session == "single-ses":
        raise Exception("not supported yet!")
elif which_dataset == "HBN":  # HBN data:
    if type_session == "single-ses":
        input_ds = "/cbica/projects/BABS/data/rawdata_HBN"   # datalad sibling in BABS cubic project
    elif type_session == "multi-ses":
        raise Exception("HBN is a single-ses data!")
elif which_dataset == "PNC":   # PNC data:
    assert type_session == "single-ses"
    input_ds = "/home/faird/zhaoc/data/PNC_BIDS"  # cloned from RBC github account


project_name = "test_babs_" + type_session
# Based on which dataset:
if which_dataset in ["HBN", "PNC"]:   # specific real dataset
    project_name += "_" + which_dataset
# Based on which BIDS App:
if bidsapp == "toybidsapp":
    input_cli = [["BIDS", input_ds]]
    container_name = bidsapp + "-0-0-7"
elif bidsapp in ["fmriprep", "fmriprep_anatonly", "fmriprep_sloppy"]:
    container_name = bidsapp + "-20-2-3"
elif bidsapp == "qsiprep":
    container_name = bidsapp + "-0-16-0RC3"
elif bidsapp == "fmriprepfake":
    container_name = bidsapp + "-0-1-1"
else:
    raise Exception("Invalid `flag_instance`!")

project_name += "_" + bidsapp + "_" + task_name


# Container:
container_ds = op.join(where_project, bidsapp + "-container")  
if flag_where == "local":
    # container_ds += "-docker"   # add "docker" at the end
    container_ds = op.join(where_project, "toybidsapp-container-docker")

container_config_yaml_file = op.join(where_notebooks, "eg_" + container_name
    + "_" + task_name + "_" + type_system)
if flag_where in ["cubic", "msi"]:
    container_config_yaml_file += "_" + flag_where
else:
    if type_system == "sge":
        container_config_yaml_file += "_" + "cubic"
    elif type_system == "slurm":
        container_config_yaml_file += "_" + "msi"
container_config_yaml_file += ".yaml"
print(container_config_yaml_file)
assert op.exists(container_config_yaml_file)
# container_config_yaml_file = op.join(where_notebooks, "example_container_" + flag_instance + ".yaml")

# list_sub_file = op.join(where_notebooks, "initial_sub_list_" + type_session + ".csv")
list_sub_file = None

/home/faird/zhaoc/babs/notebooks/eg_fmriprep-20-2-3_sloppy_slurm_msi.yaml


Let's pass these arguments into `babs-init` CLI:

In [5]:
cmd = "babs-init \\\n"
cmd += "\t" + "--where_project " + where_project + " \\\n"
cmd += "\t" + "--project_name " + project_name + " \\\n"
cmd += "\t" + "--input " + "BIDS" + " " + input_ds + " \\\n"
if list_sub_file is not None:
    cmd += "\t" + "--list_sub_file " + list_sub_file + " \\\n"
cmd += "\t" + "--container_ds " + container_ds + " \\\n"
cmd += "\t" + "--container_name " + container_name + " \\\n"
cmd += "\t" + "--container_config_yaml_file " + container_config_yaml_file + " \\\n"
cmd += "\t" + "--type_session " + type_session + " \\\n"
cmd += "\t" + "--type_system " + type_system

print("The command to execute:")
print(cmd)
print("WARNING: make sure you've changed `--fs-license-file` value in YAML file if you use it!!!")

The command to execute:
babs-init \
	--where_project /home/faird/zhaoc/data \
	--project_name test_babs_single-ses_PNC_fmriprep_sloppy \
	--input BIDS /home/faird/zhaoc/data/PNC_BIDS \
	--container_ds /home/faird/zhaoc/data/fmriprep-container \
	--container_name fmriprep-20-2-3 \
	--container_config_yaml_file /home/faird/zhaoc/babs/notebooks/eg_fmriprep-20-2-3_sloppy_slurm_msi.yaml \
	--type_session single-ses \
	--type_system slurm


In [ ]:
!($cmd)